In [1]:
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [3]:
rawOBPM = pd.read_csv('OBPM_results_raw.csv')
rawOBPM.shape

(4208, 8)

In [5]:
rawOBPM = rawOBPM[rawOBPM['Total Minutes'] >= 100]

In [7]:
for year in range(1997, 2025):
    temp = pd.read_csv(f'/Users/austincoffelt/Documents/WNBA_Teams/wnba_teams_{year}.csv')
    temp['Year'] = year
    temp['RelTAvgOff'] = temp['Offensive Rating'] - np.mean(temp['Offensive Rating'])
    temp['RelTAvgDef'] = np.mean(temp['Defensive Rating']) - temp['Defensive Rating']
    temp.to_csv(f'/Users/austincoffelt/Documents/WNBA_Teams/wnba_teams_{year}.csv')

In [9]:
file_paths = [f'/Users/austincoffelt/Documents/WNBA_Teams/wnba_teams_{year}.csv' for year in range(1997, 2025)]

teams = pd.concat((pd.read_csv(f) for f in file_paths), ignore_index=True)
teams = teams[['Team', 'Offensive Rating', 'Defensive Rating', 'Year', 'RelTAvgOff', 'RelTAvgDef']]
teams.loc[teams['Team'] == 'SAN ANTONIO SILVER STARS', 'Team'] = 'SAN ANTONIO STARS'
teams.head()

,Team,Offensive Rating,Defensive Rating,Year,RelTAvgOff,RelTAvgDef
0,HOUSTON COMETS,97.8,89.2,1997,6.225,2.35
1,NEW YORK LIBERTY,89.4,86.3,1997,-2.175,5.25
2,PHOENIX MERCURY,90.8,85.9,1997,-0.775,5.65
3,CHARLOTTE STING,91.8,90.2,1997,0.225,1.35
4,CLEVELAND ROCKERS,92.9,90.6,1997,1.325,0.95


In [11]:
# Position adjustment
team_avg = (
    rawOBPM.groupby(['Pos', 'Year'])
    .apply(lambda x: (x['OBPM'] * (x['Total Minutes'])).sum() / (x['Total Minutes']).sum())
    .rename('Pos_Avg_OBPM')
)
rawOBPM = rawOBPM.merge(team_avg, on=['Pos', 'Year'])
rawOBPM['Pos_Adj_OBPM'] = rawOBPM['OBPM'] - rawOBPM['Pos_Avg_OBPM']

In [13]:
#team adjustment
rawOBPM['Minutes_Share'] = rawOBPM['Total Minutes'] / (rawOBPM.groupby(['Team', 'Year'])['Total Minutes'].transform('sum') / 5)
rawOBPM['Off_contribution'] = rawOBPM['Pos_Adj_OBPM'] * rawOBPM['Minutes_Share']
rawOBPM['total_team_off_contribution'] = rawOBPM.groupby(['Team', 'Year'])['Off_contribution'].transform('sum')

In [15]:
map = {'HOU':'HOUSTON COMETS',
 'NYL':'NEW YORK LIBERTY',
 'PHO':'PHOENIX MERCURY',
 'CHA':'CHARLOTTE STING',
 'CLE':'CLEVELAND ROCKERS',
 'LAS':'LOS ANGELES SPARKS',
 'SAC':'SACRAMENTO MONARCHS',
 'UTA':'UTAH STARZZ',
 'DET':'DETROIT SHOCK',
 'WAS':'WASHINGTON MYSTICS',
 'MIN':'MINNESOTA LYNX',
 'ORL':'ORLANDO MIRACLE',
 'MIA':'MIAMI SOL',
 'POR':'PORTLAND FIRE',
 'IND':'INDIANA FEVER',
 'SEA':'SEATTLE STORM',
 'CON':'CONNECTICUT SUN',
 'CHI':'CHICAGO SKY',
 'ATL':'ATLANTA DREAM',
 'TUL':'TULSA SHOCK',
 'SAN':'SAN ANTONIO STARS',
 'DAL':'DALLAS WINGS',
 'LVA':'LAS VEGAS ACES'}

rawOBPM['Team'] = rawOBPM['Team'].map(map)
rawOBPM = pd.merge(teams, rawOBPM, on = ['Team', 'Year'], how = 'inner')
rawOBPM['team_adjustment'] = ((rawOBPM['RelTAvgOff'] - rawOBPM['total_team_off_contribution']) / 5)
rawOBPM['Final_OBPM'] = rawOBPM['Pos_Adj_OBPM'] + rawOBPM['team_adjustment']

In [17]:
rawOBPM = rawOBPM.sort_values('Final_OBPM', ascending = False)
rawOBPM.head(15)

,Team,Offensive Rating,Defensive Rating,Year,RelTAvgOff,RelTAvgDef,Player,Pos,Archetype,OBPM,mean,Total Minutes,Pos_Avg_OBPM,Pos_Adj_OBPM,Minutes_Share,Off_contribution,total_team_off_contribution,team_adjustment,Final_OBPM
1628,MINNESOTA LYNX,100.3,103.5,2009,1.907692,-5.053846,SEIMONE AUGUSTUS,SG,Perimeter-Oriented Big/Wing,7.331248,15.740648,178.2,-2.122742,9.453990,0.129540,1.224667,3.345263,-0.287514,9.166476
2825,WASHINGTON MYSTICS,112.9,98.1,2019,14.208333,0.608333,ELENA DELLE DONNE,SG,Perimeter-Oriented Big/Wing,8.175707,17.711563,902.1,-1.309364,9.485071,0.674629,6.398901,19.052246,-0.968783,8.516288
1147,SEATTLE STORM,100.6,98.2,2006,3.807143,-1.342857,LAUREN JACKSON,PowerF,Perimeter-Oriented Big/Wing,5.290592,11.600106,852.0,-3.148309,8.438902,0.618368,5.218348,3.668102,0.027808,8.466710
2614,WASHINGTON MYSTICS,104.2,103.5,2017,1.908333,-1.233333,ELENA DELLE DONNE,SG,Perimeter-Oriented Big/Wing,6.959211,6.299293,757.5,-1.027220,7.986431,0.557789,4.454744,0.690723,0.243522,8.229953
3004,CHICAGO SKY,105.7,102.7,2020,3.325000,-0.316667,COURTNEY VANDERSLOOT,G,Primary On-Ball Creator,6.855502,6.342414,693.0,-1.509527,8.365029,0.869532,7.273665,7.593685,-0.853737,7.511292
1846,PHOENIX MERCURY,107.3,103.7,2011,8.458333,-4.825000,PENNY TAYLOR,F,Primary On-Ball Creator,3.697691,5.838360,864.2,-3.038321,6.736012,0.656986,4.425469,4.843561,0.722954,7.458967
2982,MINNESOTA LYNX,106.7,101.7,2020,4.325000,0.683333,SYLVIA FOWLES,C,Interior Primary Option,3.557846,-0.426522,168.7,-3.186103,6.743949,0.196707,1.326583,2.665405,0.331919,7.075869
1920,CONNECTICUT SUN,102.6,97.1,2012,3.775000,1.725000,KARA LAWSON,G,Primary On-Ball Creator,4.312443,7.452878,1067.6,-2.608335,6.920778,0.769774,5.327437,3.126155,0.129769,7.050547
0,HOUSTON COMETS,97.8,89.2,1997,6.225000,2.350000,CYNTHIA COOPER,G,Primary On-Ball Creator,2.354052,5.363735,980.0,-4.732626,7.086678,0.899314,6.373146,6.512269,-0.057454,7.029224
1300,SEATTLE STORM,102.4,99.1,2007,5.953846,-2.630769,LAUREN JACKSON,PowerF,Perimeter-Oriented Big/Wing,3.815838,7.100464,1019.9,-3.130313,6.946151,0.761870,5.292064,5.818675,0.027034,6.973186


In [19]:
careerOBPM = rawOBPM.groupby('Player')['Final_OBPM'].mean()
careerOBPM = careerOBPM.sort_values(ascending = False)
careerOBPM.head(15)

Player
ELENA DELLE DONNE           6.189071
CYNTHIA COOPER              5.968760
GILLIAN GORING              5.759483
MAYA MOORE                  4.495981
PENNY TAYLOR                4.474024
BREANNA STEWART             4.160204
JONQUEL JONES               4.065571
MICHELLE CLEARY             4.003910
TAMIKA CATCHINGS            3.805176
BRANDY REED                 3.481784
JACLYN JOHNSON              3.448078
DIANA TAURASI               3.250263
NNEKA OGWUMIKE              3.241834
CLARISSA DAVIS-WRIGHTSIL    3.087148
LINDSAY WHALEN              3.067568
Name: Final_OBPM, dtype: float64

In [21]:
rawDBPM = pd.read_csv('DBPM_results_raw.csv')
rawDBPM.shape

(4322, 10)

In [23]:
rawDBPM = rawDBPM[rawDBPM['Total Minutes'] >= 100]

In [25]:
# Position adjustment
team_avg = (
    rawDBPM.groupby(['Pos', 'Year'])
    .apply(lambda x: (x['DBPM'] * (x['Total Minutes'])).sum() / (x['Total Minutes']).sum())
    .rename('Pos_Avg_DBPM')
)
rawDBPM = rawDBPM.merge(team_avg, on=['Pos', 'Year'])
rawDBPM['Pos_Adj_DBPM'] = rawDBPM['DBPM'] - rawDBPM['Pos_Avg_DBPM']

In [27]:
rawDBPM['Minutes_Share'] = rawDBPM['Total Minutes'] / (rawDBPM.groupby(['Team', 'Year'])['Total Minutes'].transform('sum') / 5)
rawDBPM['Def_contribution'] = rawDBPM['Pos_Adj_DBPM'] * rawDBPM['Minutes_Share']
rawDBPM['total_team_def_contribution'] = rawDBPM.groupby(['Team', 'Year'])['Def_contribution'].transform('sum')

In [29]:
map = {'HOU':'HOUSTON COMETS',
 'NYL':'NEW YORK LIBERTY',
 'PHO':'PHOENIX MERCURY',
 'CHA':'CHARLOTTE STING',
 'CLE':'CLEVELAND ROCKERS',
 'LAS':'LOS ANGELES SPARKS',
 'SAC':'SACRAMENTO MONARCHS',
 'UTA':'UTAH STARZZ',
 'DET':'DETROIT SHOCK',
 'WAS':'WASHINGTON MYSTICS',
 'MIN':'MINNESOTA LYNX',
 'ORL':'ORLANDO MIRACLE',
 'MIA':'MIAMI SOL',
 'POR':'PORTLAND FIRE',
 'IND':'INDIANA FEVER',
 'SEA':'SEATTLE STORM',
 'CON':'CONNECTICUT SUN',
 'CHI':'CHICAGO SKY',
 'ATL':'ATLANTA DREAM',
 'TUL':'TULSA SHOCK',
 'SAN':'SAN ANTONIO STARS',
 'DAL':'DALLAS WINGS',
 'LVA':'LAS VEGAS ACES'}

rawDBPM['Team'] = rawDBPM['Team'].map(map)
rawDBPM = pd.merge(teams, rawDBPM, on = ['Team', 'Year'], how = 'inner')
rawDBPM['team_adjustment'] = ((rawDBPM['RelTAvgDef'] - rawDBPM['total_team_def_contribution']) / 5)
rawDBPM['Final_DBPM'] = rawDBPM['Pos_Adj_DBPM'] + rawDBPM['team_adjustment']

In [31]:
rawDBPM = rawDBPM.sort_values('Final_DBPM', ascending = False)
rawDBPM.head(15)

,Team,Offensive Rating,Defensive Rating,Year,RelTAvgOff,RelTAvgDef,Player,Pos,Archetype,DBPM,...,Total Minutes,BPM,RAPM,Pos_Avg_DBPM,Pos_Adj_DBPM,Minutes_Share,Def_contribution,total_team_def_contribution,team_adjustment,Final_DBPM
2230,NEW YORK LIBERTY,89.4,98.5,2013,-7.858333,-1.200000,SAMANTHA PRAHALIS,G,Secondary Creator/Distributor,10.419664,...,108.9,-9.937182,-8.936919,-0.978280,11.397945,0.074827,0.852872,-2.170686,0.194137,11.592082
3343,WASHINGTON MYSTICS,101.1,96.0,2022,-0.650000,5.733333,EVINA WESTBROOK,G,Secondary Creator/Distributor,7.612250,...,204.0,10.641846,0.909696,-1.422375,9.034624,0.137840,1.245330,1.239362,0.898794,9.933419
2287,ATLANTA DREAM,97.6,95.5,2014,-1.350000,3.516667,DELISHA MILTON-JONES,F,Secondary Creator/Distributor,6.619744,...,342.3,-4.307863,-6.134138,-0.677760,7.297504,0.255593,1.865189,4.127754,-0.122218,7.175287
3514,LOS ANGELES SPARKS,98.0,100.0,2023,-4.308333,2.375000,EVINA WESTBROOK,G,Secondary Creator/Distributor,5.527237,...,127.8,-1.506787,14.934833,-1.607790,7.135027,0.092542,0.660287,2.288225,0.017355,7.152382
972,PHOENIX MERCURY,98.4,95.2,2004,2.438462,0.807692,ASHLEY ROBINSON,C,Roller/Cutter,7.641613,...,125.4,0.703979,3.152827,0.666351,6.975261,0.090124,0.628637,2.879825,-0.414426,6.560835
2987,SEATTLE STORM,96.2,96.4,2019,-2.491667,2.308333,BLAKE DIETRICK,G,Secondary Creator/Distributor,5.203070,...,105.4,-3.398538,-9.094020,-1.445608,6.648678,0.077783,0.517151,2.885525,-0.115438,6.533240
1046,SACRAMENTO MONARCHS,97.4,87.7,2005,1.253846,8.484615,ERIN PERPEROGLOU,F,Roller/Cutter,4.619567,...,207.0,1.501972,-5.279828,-0.744066,5.363632,0.155831,0.835821,3.415099,1.013903,6.377535
1774,WASHINGTON MYSTICS,100.0,95.8,2010,-0.466667,4.608333,JACINTA MONROE,F,Roller/Cutter,4.188664,...,117.3,-1.434422,-17.801968,-0.945123,5.133787,0.085617,0.439538,-1.591550,1.239977,6.373764
830,CONNECTICUT SUN,97.9,98.5,2003,1.014286,-1.628571,COURTNEY COLEMAN,F,Roller/Cutter,6.058361,...,142.0,0.049979,-5.975948,-0.434964,6.493325,0.128789,0.836268,-0.394977,-0.246719,6.246606
3243,LOS ANGELES SPARKS,92.3,97.5,2021,-8.175000,2.983333,LAUREN COX,F,Roller/Cutter,4.583129,...,304.2,0.171019,-10.776634,-1.138810,5.721939,0.284023,1.625162,1.335315,0.329604,6.051543


In [33]:
careerDBPM = rawDBPM.groupby('Player')['Final_DBPM'].mean()
careerDBPM = careerDBPM.sort_values(ascending = False)
careerDBPM.head(15)

Player
EVINA WESTBROOK         8.542900
JACINTA MONROE          6.373764
COURTNEY COLEMAN        6.246606
KHADIJAH WHITTINGTON    4.892710
SAMANTHA PRAHALIS       4.885957
CAMERON BRINK           4.445139
TIA JACKSON             4.236696
JENNIFER HAMSON         4.087676
LEIGHA BROWN            3.816423
TOCCARA WILLIAMS        3.781909
EUGENIA RYCRAW          3.729324
KERRI GARDIN            3.704105
KATASHA ARTIS           3.673853
OLIVIA ÉPOUPA           3.662704
AMBER HALL              3.606961
Name: Final_DBPM, dtype: float64

In [35]:
final_BPM = rawDBPM.merge(rawOBPM, on = ['Player', 'Team', 'Year'], how = 'outer')
final_BPM = final_BPM[['Player', 'Team', 'Year', 'Pos_x', 'Archetype_x', 'Minutes_Share_x', 'Final_OBPM', 'Final_DBPM']]

In [37]:
final_BPM['BPM'] = final_BPM['Final_OBPM'] + final_BPM['Final_DBPM']
final_BPM.head()

,Player,Team,Year,Pos_x,Archetype_x,Minutes_Share_x,Final_OBPM,Final_DBPM,BPM
0,A'DIA MATHIES,LOS ANGELES SPARKS,2013,G,Secondary Creator/Distributor,0.200724,-3.359363,1.187630,-2.171733
1,A'JA WILSON,LAS VEGAS ACES,2018,C,Perimeter-Oriented Big/Wing,0.768353,1.578321,-1.913276,-0.334955
2,A'JA WILSON,LAS VEGAS ACES,2019,C,Perimeter-Oriented Big/Wing,0.541246,-0.448478,-1.254321,-1.702799
3,A'JA WILSON,LAS VEGAS ACES,2020,C,Perimeter-Oriented Big/Wing,0.804216,3.080726,-0.960509,2.120217
4,A'JA WILSON,LAS VEGAS ACES,2021,C,Perimeter-Oriented Big/Wing,0.741656,1.809134,-0.250584,1.558550


In [39]:
careerBPM = final_BPM.groupby('Player')['BPM'].mean()
careerBPM = careerDBPM.sort_values(ascending = False)
careerBPM.head(15)

Player
EVINA WESTBROOK         8.542900
JACINTA MONROE          6.373764
COURTNEY COLEMAN        6.246606
KHADIJAH WHITTINGTON    4.892710
SAMANTHA PRAHALIS       4.885957
CAMERON BRINK           4.445139
TIA JACKSON             4.236696
JENNIFER HAMSON         4.087676
LEIGHA BROWN            3.816423
TOCCARA WILLIAMS        3.781909
EUGENIA RYCRAW          3.729324
KERRI GARDIN            3.704105
KATASHA ARTIS           3.673853
OLIVIA ÉPOUPA           3.662704
AMBER HALL              3.606961
Name: Final_DBPM, dtype: float64

In [41]:
final_BPM['VOAOP'] = (final_BPM['Final_OBPM'] - np.mean(final_BPM['Final_OBPM'])) * final_BPM['Minutes_Share_x']
final_BPM['VOADP'] = (final_BPM['Final_DBPM'] - np.mean(final_BPM['Final_DBPM'])) * final_BPM['Minutes_Share_x']
final_BPM['VOAP'] = (final_BPM['BPM'] - np.mean(final_BPM['BPM'])) * final_BPM['Minutes_Share_x']
final_BPM.head()

,Player,Team,Year,Pos_x,Archetype_x,Minutes_Share_x,Final_OBPM,Final_DBPM,BPM,VOAOP,VOADP,VOAP
0,A'DIA MATHIES,LOS ANGELES SPARKS,2013,G,Secondary Creator/Distributor,0.200724,-3.359363,1.187630,-2.171733,-0.524985,0.213805,-0.310583
1,A'JA WILSON,LAS VEGAS ACES,2018,C,Perimeter-Oriented Big/Wing,0.768353,1.578321,-1.913276,-0.334955,1.784293,-1.564168,0.222414
2,A'JA WILSON,LAS VEGAS ACES,2019,C,Perimeter-Oriented Big/Wing,0.541246,-0.448478,-1.254321,-1.702799,0.159901,-0.745180,-0.583666
3,A'JA WILSON,LAS VEGAS ACES,2020,C,Perimeter-Oriented Big/Wing,0.804216,3.080726,-0.960509,2.120217,3.075833,-0.870945,2.207283
4,A'JA WILSON,LAS VEGAS ACES,2021,C,Perimeter-Oriented Big/Wing,0.741656,1.809134,-0.250584,1.558550,1.893480,-0.276674,1.619015


In [43]:
highest_career_voaop = final_BPM.groupby('Player')['VOAOP'].sum()
highest_career_voaop = highest_career_voaop.sort_values(ascending = False)
highest_career_voaop.head(15)

Player
DIANA TAURASI           56.279637
TAMIKA CATCHINGS        49.889690
SUE BIRD                44.944399
LINDSAY WHALEN          40.165645
ELENA DELLE DONNE       36.295713
NNEKA OGWUMIKE          36.164954
COURTNEY VANDERSLOOT    33.951159
MAYA MOORE              32.518696
LAUREN JACKSON          31.213980
BREANNA STEWART         30.816168
PENNY TAYLOR            30.415957
BECKY HAMMON            29.089406
CANDACE PARKER          28.504717
TINA THOMPSON           28.094010
LISA LESLIE             25.466928
Name: VOAOP, dtype: float64

In [45]:
career_avg_voaop = final_BPM.groupby('Player')['VOAOP'].mean()
career_avg_voaop = career_avg_voaop.sort_values(ascending = False)
career_avg_voaop.head(15)

Player
CYNTHIA COOPER       5.006990
MAYA MOORE           4.064837
ELENA DELLE DONNE    4.032857
BREANNA STEWART      3.852021
TAMIKA CATCHINGS     3.325979
DIANA TAURASI        2.962086
JONQUEL JONES        2.901097
A'JA WILSON          2.885345
LAUREN JACKSON       2.837635
CAITLIN CLARK        2.796379
NNEKA OGWUMIKE       2.781920
ALIYAH BOSTON        2.774781
PENNY TAYLOR         2.765087
LINDSAY WHALEN       2.677710
NAPHEESA COLLIER     2.664009
Name: VOAOP, dtype: float64

In [47]:
highest_career_voadp = final_BPM.groupby('Player')['VOADP'].sum()
highest_career_voadp = highest_career_voadp.sort_values(ascending = False)
highest_career_voadp.head(15)

Player
TAMIKA CATCHINGS       22.735281
TICHA PENICHEIRO       19.843502
SHERYL SWOOPES         16.526997
KIAH STOKES            11.688582
TERESA WEATHERSPOON    11.314697
CANDACE PARKER         10.891044
JONQUEL JONES          10.447572
SANCHO LYTTLE          10.407257
ARMINTIE HERRINGTON    10.296853
ALANA BEARD             9.524014
RUTH RILEY              9.088300
MARGO DYDEK             8.699927
VICKY BULLETT           8.464806
TULLY BEVILAQUA         8.246559
TANISHA WRIGHT          7.904076
Name: VOADP, dtype: float64

In [49]:
career_avg_voadp = final_BPM.groupby('Player')['VOADP'].mean()
career_avg_voadp = career_avg_voadp.sort_values(ascending = False)
career_avg_voadp.head(15)

Player
TERESA WEATHERSPOON    1.616385
UMEKI WEBB             1.536523
TAMIKA CATCHINGS       1.515685
VICKY BULLETT          1.410801
SHERYL SWOOPES         1.377250
MERY ANDRADE           1.375175
TICHA PENICHEIRO       1.322900
JONQUEL JONES          1.305947
KIAH STOKES            1.298731
SONJA HENNING          1.284066
KEITHA DICKERSON       1.261412
KIM PERROT             1.248907
TIA JACKSON            1.160278
SUE WICKS              1.149977
ARMINTIE HERRINGTON    1.144095
Name: VOADP, dtype: float64

In [51]:
highest_career_voap = final_BPM.groupby('Player')['VOAP'].sum()
highest_career_voap = highest_career_voap.sort_values(ascending = False)
highest_career_voap.head(15)

Player
TAMIKA CATCHINGS        72.656776
LINDSAY WHALEN          42.663172
CANDACE PARKER          39.424458
LAUREN JACKSON          36.691220
COURTNEY VANDERSLOOT    36.673738
ELENA DELLE DONNE       36.582162
MAYA MOORE              36.069690
NNEKA OGWUMIKE          35.578743
BREANNA STEWART         35.088558
SUE BIRD                35.013063
DIANA TAURASI           33.818605
JONQUEL JONES           33.671256
TICHA PENICHEIRO        33.637803
SHERYL SWOOPES          33.506644
LISA LESLIE             32.572308
Name: VOAP, dtype: float64

In [53]:
career_avg_voap = final_BPM.groupby('Player')['VOAP'].mean()
career_avg_voap = career_avg_voap.sort_values(ascending = False)
career_avg_voap.head(15)

Player
TAMIKA CATCHINGS       4.843785
MAYA MOORE             4.508711
BREANNA STEWART        4.386070
CYNTHIA COOPER         4.315677
JONQUEL JONES          4.208907
ELENA DELLE DONNE      4.064685
TERESA WEATHERSPOON    3.840435
LAUREN JACKSON         3.335565
NAPHEESA COLLIER       3.128919
SHERYL SWOOPES         3.046059
LISA LESLIE            2.961119
VICKY BULLETT          2.929489
MICHELLE CLEARY        2.844515
LINDSAY WHALEN         2.844211
KIM PERROT             2.777508
Name: VOAP, dtype: float64

In [55]:
final_BPM[final_BPM['Player'] == "BREANNA STEWART"]

,Player,Team,Year,Pos_x,Archetype_x,Minutes_Share_x,Final_OBPM,Final_DBPM,BPM,VOAOP,VOADP,VOAP
431,BREANNA STEWART,NEW YORK LIBERTY,2023,F,Perimeter-Oriented Big/Wing,0.934067,5.328756,0.692066,6.020821,5.672279,0.532045,6.207106
432,BREANNA STEWART,NEW YORK LIBERTY,2024,F,Perimeter-Oriented Big/Wing,0.788412,3.521912,1.186279,4.708191,3.363224,0.838723,4.204296
433,BREANNA STEWART,SEATTLE STORM,2016,F,Perimeter-Oriented Big/Wing,0.877880,1.845219,0.530457,2.375676,2.272944,0.358168,2.633726
434,BREANNA STEWART,SEATTLE STORM,2017,F,Perimeter-Oriented Big/Wing,0.793838,3.909953,0.468703,4.378656,3.694412,0.274856,3.971632
435,BREANNA STEWART,SEATTLE STORM,2018,F,Perimeter-Oriented Big/Wing,0.780712,5.738825,0.209909,5.948734,5.061148,0.068268,5.131741
436,BREANNA STEWART,SEATTLE STORM,2020,F,Perimeter-Oriented Big/Wing,0.704862,2.953012,1.694339,4.647351,2.605820,1.107955,3.715874
437,BREANNA STEWART,SEATTLE STORM,2021,F,Perimeter-Oriented Big/Wing,0.732409,4.087187,0.500258,4.587445,3.538337,0.276698,3.817217
438,BREANNA STEWART,SEATTLE STORM,2022,F,Perimeter-Oriented Big/Wing,0.693905,5.896772,1.270886,7.167658,4.608003,0.796895,5.406964


In [57]:
df2024 = final_BPM[final_BPM['Year'] == 2024]
df2024 = df2024.sort_values('VOAP', ascending = False)
df2024.head(15)

,Player,Team,Year,Pos_x,Archetype_x,Minutes_Share_x,Final_OBPM,Final_DBPM,BPM,VOAOP,VOADP,VOAP
7,A'JA WILSON,LAS VEGAS ACES,2024,C,Perimeter-Oriented Big/Wing,0.822262,6.056217,-0.218374,5.837843,5.591489,-0.280259,5.313679
1559,JONQUEL JONES,NEW YORK LIBERTY,2024,C,Interior Primary Option,0.739874,2.967169,2.389440,5.356608,2.745729,1.677275,4.425208
432,BREANNA STEWART,NEW YORK LIBERTY,2024,F,Perimeter-Oriented Big/Wing,0.788412,3.521912,1.186279,4.708191,3.363224,0.838723,4.204296
76,ALANNA SMITH,MINNESOTA LYNX,2024,F,Secondary Creator/Distributor,0.605682,2.106559,2.978445,5.085003,1.726479,1.729816,3.458099
2445,NAPHEESA COLLIER,MINNESOTA LYNX,2024,F,Perimeter-Oriented Big/Wing,0.691421,2.771621,1.118770,3.890391,2.430713,0.688866,3.121639
3051,SKYLAR DIGGINS,SEATTLE STORM,2024,G,Primary On-Ball Creator,0.798575,2.375735,0.827506,3.203241,2.491270,0.563028,3.056676
1740,KAYLA MCBRIDE,MINNESOTA LYNX,2024,G,Spot-up Shooter,0.724533,0.539952,2.619105,3.159057,0.930201,1.808898,2.741257
2572,NNEKA OGWUMIKE,SEATTLE STORM,2024,F,Perimeter-Oriented Big/Wing,0.743357,2.734096,0.226734,2.960830,2.585400,0.077509,2.665122
1349,JACKIE YOUNG,LAS VEGAS ACES,2024,G,Primary On-Ball Creator,0.758731,2.654891,0.202979,2.857871,2.578776,0.061088,2.642124
184,ALYSSA THOMAS,CONNECTICUT SUN,2024,F,Interior Primary Option,0.772247,1.467468,1.223682,2.691150,1.707730,0.850411,2.560441


In [59]:
df2024 = df2024.sort_values('VOAOP', ascending = False)
df2024.head(15)

,Player,Team,Year,Pos_x,Archetype_x,Minutes_Share_x,Final_OBPM,Final_DBPM,BPM,VOAOP,VOADP,VOAP
7,A'JA WILSON,LAS VEGAS ACES,2024,C,Perimeter-Oriented Big/Wing,0.822262,6.056217,-0.218374,5.837843,5.591489,-0.280259,5.313679
297,ARIKE OGUNBOWALE,DALLAS WINGS,2024,G,Primary On-Ball Creator,1.013375,2.698617,-2.418774,0.279843,3.488572,-2.575229,0.916361
432,BREANNA STEWART,NEW YORK LIBERTY,2024,F,Perimeter-Oriented Big/Wing,0.788412,3.521912,1.186279,4.708191,3.363224,0.838723,4.204296
1825,KELSEY MITCHELL,INDIANA FEVER,2024,G,Primary On-Ball Creator,0.845588,2.637947,-2.716267,-0.078319,2.859657,-2.400397,0.461779
528,CAITLIN CLARK,INDIANA FEVER,2024,G,Primary On-Ball Creator,0.935431,2.245490,-1.657763,0.587727,2.796379,-1.665282,1.133883
1559,JONQUEL JONES,NEW YORK LIBERTY,2024,C,Interior Primary Option,0.739874,2.967169,2.389440,5.356608,2.745729,1.677275,4.425208
2572,NNEKA OGWUMIKE,SEATTLE STORM,2024,F,Perimeter-Oriented Big/Wing,0.743357,2.734096,0.226734,2.960830,2.585400,0.077509,2.665122
1349,JACKIE YOUNG,LAS VEGAS ACES,2024,G,Primary On-Ball Creator,0.758731,2.654891,0.202979,2.857871,2.578776,0.061088,2.642124
3051,SKYLAR DIGGINS,SEATTLE STORM,2024,G,Primary On-Ball Creator,0.798575,2.375735,0.827506,3.203241,2.491270,0.563028,3.056676
2445,NAPHEESA COLLIER,MINNESOTA LYNX,2024,F,Perimeter-Oriented Big/Wing,0.691421,2.771621,1.118770,3.890391,2.430713,0.688866,3.121639


In [61]:
final_BPM[final_BPM['Player'] == "KELSEY PLUM"]

,Player,Team,Year,Pos_x,Archetype_x,Minutes_Share_x,Final_OBPM,Final_DBPM,BPM,VOAOP,VOADP,VOAP
1826,KELSEY PLUM,LAS VEGAS ACES,2018,G,Spot-up Shooter,0.601488,2.606908,0.086685,2.693593,2.015478,-0.021521,1.995748
1827,KELSEY PLUM,LAS VEGAS ACES,2019,G,Spot-up Shooter,0.635509,-1.004247,0.401878,-0.602369,-0.165447,0.177569,0.014016
1828,KELSEY PLUM,LAS VEGAS ACES,2021,G,Primary On-Ball Creator,0.481698,4.716200,-0.174988,4.541212,2.630126,-0.143283,2.488278
1829,KELSEY PLUM,LAS VEGAS ACES,2022,G,Primary On-Ball Creator,0.817276,3.488383,-1.623202,1.865181,3.458950,-1.426691,2.034693
1830,KELSEY PLUM,LAS VEGAS ACES,2023,G,Primary On-Ball Creator,0.793210,2.754298,-0.352419,2.401879,2.774815,-0.376683,2.400494
1831,KELSEY PLUM,LAS VEGAS ACES,2024,G,Primary On-Ball Creator,0.812701,0.573876,-1.059400,-0.485524,1.070967,-0.960503,0.112884
1832,KELSEY PLUM,SAN ANTONIO STARS,2017,G,Primary On-Ball Creator,0.540506,-2.454266,-1.649956,-4.104222,-0.924457,-0.958004,-1.880851
